In [1]:
#import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

df_diet = pd.read_csv("data/diet.csv")
display(df_diet)

df_recipes = pd.read_csv("data/recipes.csv")
display(df_recipes)

df_requests = pd.read_csv("data/requests.csv")
display(df_requests)

df_reviews = pd.read_csv("data/reviews.csv")
display(df_reviews)

,AuthorId,Diet,Age
0,10000120E,Vegetarian,46
1,1000014D,Vegan,18
2,1000015A,Vegetarian,58
3,1000016E,Vegetarian,32
4,1000027E,Vegan,61
...,...,...,...
271902,999917E,Vegetarian,28
271903,999936C,Omnivore,22
271904,99993D,Vegetarian,58
271905,99994A,Vegetarian,18


,RecipeId,Name,CookTime,PrepTime,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"c(""\""6\"""", ""\""2\"""", ""\""1 1/2\"""", ""\""1/4\"""", ""\...","c(""\""hazelnuts\"""", ""\""broccoli florets\"""", ""\""...",241.3,10.1,1.2,0.0,13.1,31.8,2.3,1.4,6.7,9.0,NaN
1,365718,Cashew-chutney Rice,3600,600,Other,"c(""\""1\"""", ""\""3/4\"""", ""\""6\"""", ""\""5\"""", ""\""2\""...","c(""\""celery\"""", ""\""onion\"""", ""\""butter\"""", ""\""...",370.8,17.5,7.2,22.9,553.3,44.3,1.6,2.2,9.4,8.0,NaN
2,141757,Copycat Taco Bell Nacho Fries BellGrande,3600,2700,Other,"c(""\""3\"""", ""\""1/2\"""", ""\""1\"""", ""\""1\"""", ""\""3\""...","c(""\""Copycat Taco Bell Seasoned Beef\"""", ""\""ye...",377.6,20.9,10.5,45.7,1501.8,36.6,3.8,6.1,12.9,8.0,NaN
3,280351,Slow Cooker Jalapeno Cheddar Cheese Soup,18000,1800,Other,"c(""\""2\"""", ""\""1\"""", ""\""2\"""", ""\""2\"""", ""\""1\"""",...","c(""\""unsalted butter\"""", ""\""yellow onion\"""", ""...",282.8,16.5,10.3,50.5,630.2,22.8,2.3,2.7,11.7,6.0,NaN
4,180505,Cool & Crisp Citrus Chiffon Pie,3600,1800,Other,"c(""\""1\"""", ""\""1/4\"""", ""\""1/2\"""", ""\""1/2\"""", ""\...","c(""\""unflavored gelatin\"""", ""\""water\"""", ""\""su...",257.5,8.6,2.4,110.7,160.9,39.8,0.4,30.2,6.3,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75599,253577,Frijoles Negros- Crock Pot Mexican Black Beans,43200,28800,Other,"c(""\""2\"""", ""\""6 -8\"""", ""\""5\"""", ""\""1/2\"""", ""\""...","c(""\""black beans\"""", ""\""water\"""", ""\""bay leave...",121.5,0.5,0.1,0.0,1175.1,22.2,7.8,0.6,7.9,NaN,NaN
75600,267827,Moose Moussaka,3600,2700,Other,"c(""\""1\"""", ""\""6\"""", ""\""2\"""", ""\""2\"""", ""\""1/2\""...","c(""\""onion\"""", ""\""garlic cloves\"""", ""\""olive o...",652.2,25.8,10.7,197.9,435.5,51.9,7.5,7.2,50.1,NaN,NaN
75601,266983,Cantonese Pepper Steak for Two (Or More),1800,900,Other,"c(""\""1/2\"""", ""\""1\"""", ""\""1/8\"""", ""\""1/8\"""", ""\...","c(""\""top round steak\"""", ""\""cornstarch\"""", ""\""...",223.9,9.2,3.6,78.3,725.9,7.3,1.1,1.7,26.7,2.0,NaN
75602,253739,Coconut Cream Cooler,300,120,Other,"c(""\""1\"""", ""\""1\"""", ""\""1\"""", ""\""7 1/2\"""", ""\""1...","c(""\""cream of coconut\"""", ""\""water\"""")",2229.8,80.3,69.3,0.0,294.7,369.0,15.7,317.9,26.7,NaN,1 gallon


,AuthorId,RecipeId,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,2001012259B,73440,1799.950949,0.0,Indifferent,0,0,0
1,437641B,365718,4201.820980,0.0,Yes,0,Indifferent,1
2,1803340263D,141757,6299.861496,0.0,Indifferent,1,Indifferent,0
3,854048B,280351,19801.365796,0.0,Yes,1,0,1
4,2277685E,180505,5400.093457,0.0,Indifferent,0,0,0
...,...,...,...,...,...,...,...,...
140190,163793B,78171,1560.649725,0.0,Indifferent,0,0,1
140191,33888B,333262,1502.011466,1.0,Indifferent,1,0,0
140192,401942C,49200,5999.274269,0.0,Indifferent,0,0,1
140193,346866B,214815,899.523513,0.0,Yes,1,Indifferent,1


C:\Users\Asus\AppData\Local\Temp\ipykernel_19372\3807219887.py:17: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reviews = pd.read_csv("data/reviews.csv")


,AuthorId,RecipeId,Rating,Like,TestSetId
0,2492191A,33671,2.0,NaN,1.0
1,2002019979A,92647,2.0,NaN,2.0
2,408594E,161770,NaN,NaN,3.0
3,2001625557E,108231,2.0,NaN,4.0
4,2001427116E,71109,NaN,NaN,5.0
...,...,...,...,...,...
140190,999595E,338070,2.0,False,NaN
140191,999774A,29002,2.0,False,NaN
140192,999774A,159252,NaN,False,NaN
140193,999774A,1171,2.0,True,NaN


"\nwith pd.ExcelWriter('/Users/fabianjanz/Downloads/analytics_cup_2324_raw_gYELgDZ/overview.xlsx', engine='xlsxwriter') as writer:\n    df_diet.to_excel(writer, sheet_name='Diet', index=False)\n    df_recipes.to_excel(writer, sheet_name='Recipes', index=False)\n    df_requests.to_excel(writer, sheet_name='Requests', index=False)\n    df_reviews.to_excel(writer, sheet_name='Reviews', index=False)\n"

In [2]:
# Load data
recipes = df_recipes
reviews = df_reviews
diet = df_diet
requests = df_requests

prediction_on_revies = reviews.dropna(subset=["TestSetId"])
#print(prediction_on_revies)

reviews_got = reviews.dropna(subset=["Like"])
#print(reviews_got)


"""
label_encoder = LabelEncoder()
reviews['AuthorId'] = label_encoder.fit_transform(reviews['AuthorId'])
reviews['Rating'].fillna(reviews['Rating'].mean(), inplace=True)
X = reviews.drop(['Like','TestSetId'],axis=1)
y = reviews['Like']
y_train = y.dropna()
y_test = prediction_on_revies['Like']
# Step 2: Train-Test Split
X_train, X_test, a, b   = train_test_split(X, y, test_size=0.2, random_state=42)
# Step 3: Model Training
model = RandomForestClassifier()
model.fit(X_train, y_train)
# Step 4: Model Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
"""


# Merge datasets based on common keys
merged_data = pd.merge(reviews, requests, on=['AuthorId','RecipeId'], how='inner')
merged_data = pd.merge(merged_data, recipes, on='RecipeId', how='left')
merged_data = pd.merge(merged_data, diet, on='AuthorId', how='left')
#print(merged_data)

# Data preprocessing
# For simplicity, let's drop irrelevant columns and handle missing values 
merged_data = merged_data.drop(['Rating','RecipeYield','RecipeServings','TestSetId','AuthorId','RecipeId','RecipeIngredientQuantities','RecipeIngredientParts'], axis=1)
merged_data = merged_data.dropna()
print(merged_data)

# Encode categorical variables
label_encoder = LabelEncoder()
merged_data['Diet'] = label_encoder.fit_transform(merged_data['Diet'])
merged_data['Name'] = label_encoder.fit_transform(merged_data['Name'])
merged_data['RecipeCategory'] = label_encoder.fit_transform(merged_data['RecipeCategory'])
merged_data['LowSugar'] = label_encoder.fit_transform(merged_data['LowSugar'])
#merged_data['RecipeYield'] = label_encoder.fit_transform(merged_data['RecipeYield'])
merged_data['HighProtein'] = label_encoder.fit_transform(merged_data['HighProtein'])
merged_data['Like'] = label_encoder.fit_transform(merged_data['Like'])

#print(merged_data)


         Like         Time  HighCalories  HighProtein  LowFat     LowSugar  \
42814   False   119.024930           0.0  Indifferent       0  Indifferent   
42815   False  1199.386790           0.0          Yes       0            0   
42816   False   362.152341           0.0          Yes       0  Indifferent   
42817   False  1198.957497           0.0          Yes       0            0   
42818   False  5400.036634           1.0  Indifferent       0            0   
...       ...          ...           ...          ...     ...          ...   
140190  False  3899.421310           0.0  Indifferent       1  Indifferent   
140191  False  2402.372535           0.0  Indifferent       0  Indifferent   
140192  False  5999.598903           0.0          Yes       0            0   
140193   True   480.233207           1.0          Yes       0            0   
140194  False  3600.387748           0.0  Indifferent       0  Indifferent   

        HighFiber                                     Name  Coo

In [3]:
# Define features and target variable
X = merged_data.drop('Like', axis=1)
y = merged_data['Like']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier (you can choose a different classifier based on your preferences)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
balanced_acc = balanced_accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Balanced Accuracy: {balanced_acc:.8f}')
print('Classification Report:')
print(report)
print(y_pred)

# Now, you can use the trained model to make predictions on the private test set for submission
# Ensure to preprocess the private test set similarly before making predictions

# Display or save the predictions
private_test_predictions = pd.DataFrame({'True_Label': y_test, 'Predicted_Label': y_pred})
private_test_predictions.to_csv('data/Results.csv', index=False)

Balanced Accuracy: 0.64628680
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     16813
           1       0.74      0.31      0.44      2664

    accuracy                           0.89     19477
   macro avg       0.82      0.65      0.69     19477
weighted avg       0.88      0.89      0.87     19477

[0 0 0 ... 0 0 0]
